# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set Interface Parameters (e.g. `distance_z`, `max_area`, `miller_indices`) below or use the default values
3. Additionally, specify whether termination is selected using interactive prompt, or the via the variable in the code.
3. Click "Run" > "Run All Cells" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [1]:
IS_TERMINATIONS_SELECTION_INTERACTIVE = False # Enable interactive selection of terminations via UI prompt

## 1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` as directed in README.

In [2]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

## 3. Load input Materials


In [3]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

from utils.visualize import visualize_materials as visualize

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[0]
film = materials[1]

Data from 0-Ni has been read successfully.
Data from 1-Graphene has been read successfully.
Data from 2-WS2 has been read successfully.
Data from 3-BN has been read successfully.
Data from 4-Te2Mo has been read successfully.
Data from 5-HfO2 has been read successfully.
Data from Hf12 C2 O24, Interface has been read successfully.
Data from Ni3 C2, Interface has been read successfully.


### 1.4. Visualize Substrate and Film

In [4]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Ni - Material', layout=Layout(align_self='center', height='30px'…

## 2. Set Interface Parameters

### 2.1. Set Substrate and Layer Parameters 
Imported `InterfaceSettings` is a class that specifies the parameters for the construction of the interface. The default values are assumed if properties are not set during the initialization.
Additionally, specify if the termination selection is done using interactive prompt, or the via selecting the termination index in the code.

In [5]:
from mat3ra.made.tools.build.slab import SlabConfiguration

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    # the Miller indices of the interfacial plane of the substrate
    miller_indices=(0,0,1),
    thickness=3,
    vacuum=3,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

film_slab_configuration = SlabConfiguration(
    bulk=film,
    # the Miller indices of the interfacial plane of the layer
    miller_indices=(0, 0, 1),
    thickness=1,
    vacuum=1,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

substrate_slab_terminations = substrate_slab_configuration.terminations
film_slab_terminations = film_slab_configuration.terminations

print(f"Substrate terminations: {substrate_slab_terminations}")
print(f"Film terminations: {film_slab_terminations}")

Substrate terminations: ['Ni_P4/mmm_1']
Film terminations: ['C_P6/mmm_2']


### 2.2. Visualize slabs for possible terminations

In [6]:
for termination in substrate_slab_terminations:
    visualize(substrate_slab_configuration.get_material(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")
    
for termination in film_slab_terminations:
    visualize(film_slab_configuration.get_material(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")
    

GridBox(children=(VBox(children=(Label(value='Ni6 - Termination: Ni_P4/mmm_1', layout=Layout(align_self='cente…

GridBox(children=(VBox(children=(Label(value='C2 - Termination: C_P6/mmm_2', layout=Layout(align_self='center'…

### 2.3. Select the termination pair for the interface
Select termination that will be facing interfacial plane for substrate and film

In [7]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

substrate_termination = substrate_slab_terminations[0]
film_termination = film_slab_terminations[0]

if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        substrate_termination = await ui_prompt_select_array_element_by_index_pyodide(substrate_slab_terminations, element_name="substrate termination")
        film_termination = await ui_prompt_select_array_element_by_index_pyodide(film_slab_terminations, element_name="film termination")
    else:
        substrate_termination = ui_prompt_select_array_element_by_index(substrate_slab_terminations, element_name="substrate termination")
        film_termination = ui_prompt_select_array_element_by_index(film_slab_terminations, element_name="film termination")

termination_pair = (film_termination, substrate_termination)
print(termination_pair)
# termination_pair = ('C_P6/mmm_2', 'Ni_R-3m_1')

('C_P6/mmm_2', 'Ni_P4/mmm_1')


### 2.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [8]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
zsl_strain_matching_parameters = ZSLStrainMatchingParameters()

## 4. Create interfaces

### 4.1. Initialize the interface builder

Initialize the interface builder with the materials and interface settings.

In [11]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration, ZSLInterfaceBuilder

interface_configuration = InterfaceConfiguration(substrate_slab_configuration, film_slab_configuration, termination_pair, distance_z=3.0, vacuum=5.0)

matched_interfaces_builder = ZSLInterfaceBuilder(strain_matching_parameters=zsl_strain_matching_parameters)

interfaces = matched_interfaces_builder.get_materials(configuration=interface_configuration)

interfaces = list(interfaces)
print(interfaces)

[<mat3ra.made.material.Material object at 0x139ca61d0>, <mat3ra.made.material.Material object at 0x13a239f50>, <mat3ra.made.material.Material object at 0x13a268910>, <mat3ra.made.material.Material object at 0x13a6b4810>, <mat3ra.made.material.Material object at 0x1378076d0>, <mat3ra.made.material.Material object at 0x13a640bd0>]


### 4.4. Print out interface with the lowest strain for selected termination

In [14]:
print(f"Interface with lowest strain for termination {termination_pair} (index 0):")
first_interface = interfaces[0]
# print(f"    strain: {first_interface.get_mean_abs_strain() * 100:.3f}%")
# print("    number of atoms:", first_interface.num_sites)

Interface with lowest strain for termination ('C_P6/mmm_2', 'Ni_P4/mmm_1') (index 0):


## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [16]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces, PLOT_SETTINGS)

ImportError: cannot import name 'InterfaceDataHolder' from 'mat3ra.made.tools.build.interface' (/Users/mat3ra/code/GREEN/api-examples/.venv/lib/python3.11/site-packages/mat3ra/made/tools/build/interface.py)

## 6. Select the interface to pass outside this kernel

### 6.1. Select the interface with the desired termination and strain

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [ ]:
# Could be either the termination as tuple, e.g. `('Ni_P6/mmm_1', 'C_C2/m_2')` or its index: `0`
termination_or_its_index = selected_termination
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = slice(0, 1)
selected_interfaces = interface_data_holder.get_interfaces_as_materials(termination_or_its_index, interfaces_slice_range_or_index)

### 6.2. Visualize the selected interface(s)

In [ ]:
visualize(selected_interfaces, repetitions=[1, 1, 1], rotation="0x")

### 6.3. Pass data to the outside runtime
Enrich the selected interfaces names with the strain values and pass them to the application runtime.

In [ ]:
from utils.jupyterlite import set_data

for interface in selected_interfaces:
    if "Interface, Strain:" not in interface["name"]:
        interface["name"] = f'{interface["name"]}, Interface, Strain:{interface["metadata"]["interface_properties"]["mean_abs_strain"]*100:.3f}%'

set_data("materials", selected_interfaces)